# TestDataManagerを作成する準備
- 2024.10.01

In [1]:
import pandas as pd

In [2]:
# エクセルファイルの読み込み
xls = pd.ExcelFile('/Users/r/Documents/fastapi/testdata.xlsx')

class_df = pd.read_excel(xls, 'クラス', usecols=['クラス'])
subject_df = pd.read_excel(xls, '科目', usecols=['科目ID', '学年', '科目名', '単位数'])
lecture_df = pd.read_excel(xls, '講座', usecols=['講座ID', '講座名'])
block_df = pd.read_excel(xls, 'ブロック', usecols=['ブロックID', 'ブロック名'])
block_lane_df = pd.read_excel(xls, 'ブロックレーン', usecols=['ブロックID', 'レーンID', 'レーン番号'])
lane_lecture_df = pd.read_excel(xls, 'レーン講座', usecols=['レーンID', '講座ID'])
class_block_df = pd.read_excel(xls, 'クラスブロック', usecols=['クラス', 'ブロックID'])

In [3]:
lane_lecture_df

,レーンID,講座ID
0,101,j0101
1,101,j0201
2,101,g0101
3,101,m0101
4,101,m0201
...,...,...
311,3801,e0606
312,3901,s1003
313,3901,s1601
314,3902,s1102


In [4]:
# 1. レーン-講座に講座を結合
lane_lecture_df = lane_lecture_df.merge(lecture_df, left_on='講座ID', right_on='講座ID', how='left')

lane_lecture_df

,レーンID,講座ID,講座名
0,101,j0101,1現国1
1,101,j0201,1言文1
2,101,g0101,1歴総1
3,101,m0101,1数Ⅰ1
4,101,m0201,1数Ａ1
...,...,...,...
311,3801,e0606,3英表Ⅱ6
312,3901,s1003,3物理6
313,3901,s1601,3物探
314,3902,s1102,3生物6


In [5]:
# 2. ブロック-レーンにレーン-講座を結合
block_lane_lecture_df = lane_lecture_df.merge(block_lane_df, left_on='レーンID', right_on='レーンID', how='left')

block_lane_lecture_df

,レーンID,講座ID,講座名,ブロックID,レーン番号
0,101,j0101,1現国1,1,1
1,101,j0201,1言文1,1,1
2,101,g0101,1歴総1,1,1
3,101,m0101,1数Ⅰ1,1,1
4,101,m0201,1数Ａ1,1,1
...,...,...,...,...,...
311,3801,e0606,3英表Ⅱ6,38,1
312,3901,s1003,3物理6,39,1
313,3901,s1601,3物探,39,1
314,3902,s1102,3生物6,39,2


In [6]:
# 3. クラス-ブロックにブロック-レーンを結合
final_df = block_lane_lecture_df.merge(class_block_df, left_on='ブロックID', right_on='ブロックID', how='left')

In [7]:
final_df

,レーンID,講座ID,講座名,ブロックID,レーン番号,クラス
0,101,j0101,1現国1,1,1,1-1
1,101,j0201,1言文1,1,1,1-1
2,101,g0101,1歴総1,1,1,1-1
3,101,m0101,1数Ⅰ1,1,1,1-1
4,101,m0201,1数Ａ1,1,1,1-1
...,...,...,...,...,...,...
388,3801,e0606,3英表Ⅱ6,38,1,3-6
389,3901,s1003,3物理6,39,1,3-6
390,3901,s1601,3物探,39,1,3-6
391,3902,s1102,3生物6,39,2,3-6


In [8]:
# 講座名をリスト化
lane_group = final_df.groupby(['クラス', 'ブロックID', 'レーンID'])['講座名'].agg(list).reset_index()

# レーンごとにリスト化
block_group = lane_group.groupby(['クラス', 'ブロックID'])['講座名'].agg(list).reset_index()

# ブロックごとにリスト化
final_result = block_group.groupby('クラス')['講座名'].agg(list).to_dict()

print(final_result)

{'1-1': [[['1現国1', '1言文1', '1歴総1', '1数Ⅰ1', '1数Ａ1', '1物基1', '1化基1', '1生基1', '1体育1267', '1保健1', '1音楽Ⅰ1', '1英コⅠ1', '1論表Ⅰ1', '1家基1', '1情報Ⅰ1']]], '1-2': [[['1現国2', '1言文2', '1歴総2', '1数Ⅰ2', '1数Ａ2', '1物基2', '1化基2', '1生基2', '1体育1267', '1保健2', '1英コⅠ2', '1論表Ⅰ2', '1家基2', '1情報Ⅰ2']], [['1音楽Ⅰ23'], ['1美術Ⅰ23'], ['1書道Ⅰ23']]], '1-3': [[['1音楽Ⅰ23'], ['1美術Ⅰ23'], ['1書道Ⅰ23']], [['1現国3', '1言文3', '1歴総3', '1数Ⅰ3', '1数Ａ3', '1物基3', '1化基3', '1生基3', '1体育345', '1保健3', '1英コⅠ3', '1論表Ⅰ3', '1家基3', '1情報Ⅰ3']]], '1-4': [[['1現国4', '1言文4', '1歴総4', '1数Ⅰ4', '1数Ａ4', '1物基4', '1化基4', '1生基4', '1体育345', '1保健4', '1英コⅠ4', '1論表Ⅰ4', '1家基4', '1情報Ⅰ4']], [['1音楽Ⅰ45'], ['1美術Ⅰ45'], ['1書道Ⅰ45']]], '1-5': [[['1音楽Ⅰ45'], ['1美術Ⅰ45'], ['1書道Ⅰ45']], [['1現国5', '1言文5', '1歴総5', '1数Ⅰ5', '1数Ａ5', '1物基5', '1化基5', '1生基5', '1体育345', '1保健5', '1英コⅠ5', '1論表Ⅰ5', '1家基5', '1情報Ⅰ5']]], '1-6': [[['1現国6', '1言文6', '1歴総6', '1数Ⅰ6', '1数Ａ6', '1物基6', '1化基6', '1生基6', '1体育1267', '1保健6', '1英コⅠ6', '1論表Ⅰ6', '1家基6', '1情報Ⅰ6']], [['1音楽Ⅰ67'], ['1美術Ⅰ67'], ['1書道Ⅰ67']]], '1-7': [[['1音楽Ⅰ67

In [9]:
final_result['3-6']

[[['3現文6', '3古典理6', '3地B6', '3数Ⅲ6', '3数探Ⅲ6', '3化学6', '3化探', '3コ英Ⅲ6', '3英表Ⅱ6']],
 [['3物理6', '3物探'], ['3生物6', '3生探']]]